In [1]:
// All current dependencies. To be replaced by mdms assembly.

val pathToMDMS = "/home/lawrence/repos/metadata-ms/mdms"

classpath.addPath(s"$pathToMDMS/mdms-distro/target/mdms-distro-0.0.3-SNAPSHOT-jar-with-dependencies.jar")
classpath.addPath(s"$pathToMDMS/mdms-analytics/target/mdms-analytics-0.0.3-SNAPSHOT.jar")

classpath.add("io.continuum.bokeh" %% "bokeh" % "0.7")
classpath.add("com.typesafe.play" %% "play" % "2.5.1")

Adding 29 artifact(s)
Adding 27 artifact(s)


pathToMDMS: String = "/home/lawrence/repos/metadata-ms/mdms"

In [2]:
// Get Metadatastore from existing sqlite.

import de.hpi.isg.mdms.rdbms._
import de.hpi.isg.mdms.domain._
import de.hpi.isg.mdms.clients.util.MetadataStoreUtil
import de.hpi.isg.mdms.clients.parameters.MetadataStoreParameters

val mdsParams = new MetadataStoreParameters()
mdsParams.metadataStore = "/home/lawrence/Downloads/metadata-store"
val mds = MetadataStoreUtil.loadMetadataStore(mdsParams)
val constraintColl = mds.getConstraintCollection(1952386454)
val allColls = mds.getConstraintCollections

import de.hpi.isg.mdms.rdbms._
import de.hpi.isg.mdms.domain._
import de.hpi.isg.mdms.clients.util.MetadataStoreUtil
import de.hpi.isg.mdms.clients.parameters.MetadataStoreParameters
mdsParams: de.hpi.isg.mdms.clients.parameters.MetadataStoreParameters = de.hpi.isg.mdms.clients.parameters.MetadataStoreParameters@7778e12
mds: de.hpi.isg.mdms.model.MetadataStore = MetadataStore[SQLiteInterface[class org.sqlite.SQLiteConnection]]
constraintColl: de.hpi.isg.mdms.model.constraints.ConstraintCollection = RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=1952386454]
allColls: java.util.Collection[de.hpi.isg.mdms.model.constraints.ConstraintCollection] = [RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=333129662], RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=742620611], RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=1377315586], RDBMSConstraintCollection [scope=[Schema[musicbrainz,

In [58]:
import scala.collection.JavaConverters._
import de.hpi.isg.mdms.domain.constraints._

val cleanColls = allColls.asScala.filter(cc => !cc.getDescription.contains("FD"))
val ind = cleanColls.map(_.getConstraints.asScala.head).tail.head.asInstanceOf[InclusionDependency]
val targets = ind.getTargetReference.getReferencedColumns
val target = targets.head
val idUtils = mds.getIdUtils

val schema = idUtils.getSchemaId(target)
val table = idUtils.getTableId(target)
val column = target

val targetSchema = mds.getSchemaById(schema)
val targetTable = targetSchema.getTableById(table)
val targetColumn = targetTable.getColumnById(column)

val columnStats = constraintColl.getConstraints.asScala.filter(_.getClass == classOf[ColumnStatistics]) 
val stat = columnStats.find(_.getTargetReference.asInstanceOf[SingleTargetReference].getTargetId == target).get.asInstanceOf[ColumnStatistics]

stat.getNumDistinctValues


import scala.collection.JavaConverters._
import de.hpi.isg.mdms.domain.constraints._
cleanColls: Iterable[de.hpi.isg.mdms.model.constraints.ConstraintCollection] = List(
  RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=742620611],
  RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=1377315586],
  RDBMSConstraintCollection [scope=[Schema[musicbrainz, , 00ffffff]], getId()=1952386454]
)
ind: InclusionDependency = InclusionDependency [target=Reference [dependentColumns=[1007622, 1007623, 1007624], referencedColumns=[61444, 61447, 61448]]]
targets: Array[Int] = Array(61444, 61447, 61448)
target: Int = 61444
idUtils: de.hpi.isg.mdms.model.util.IdUtils = IdUtils [8/12/12]
schema: Int = 16777215
table: Int = 65535
column: Int = 61444
targetSchema: de.hpi.isg.mdms.model.targets.Schema = Schema[musicbrainz, , 00ffffff]
targetTable: de.hpi.isg.mdms.model.targets.Table = Table[l_area_event, 0000ffff]
targetColumn: de.hpi.isg.mdms.model.target

In [ ]:
// val inds = mds.getConstraintCollections(fancy stuff here)
// val distinctValues = mds.getConstraintCollections(fancy stuff here)

val indDistribution = inds
    .join(dvs).where(ind => ind.lhs).equal(dv => dv.column)
    .join(dvs).where(ind => ind.rhs).equal(dv => dv.column)
    .groupBy(...)
    .aggregate("count")

In [30]:
import de.hpi.isg.mdms.model.util.BasicPrettyPrinter

val pp = new BasicPrettyPrinter(mds)
pp.prettyPrint(target)

import de.hpi.isg.mdms.model.util.BasicPrettyPrinter
pp: BasicPrettyPrinter = de.hpi.isg.mdms.model.util.BasicPrettyPrinter@7c5ca315
res29_2: String = "musicbrainz.l_area_event.[4]"

In [3]:
// import scala.collection.JavaConverters._
import de.hpi.isg.mdms.analytics.ConstraintImplicits._
import de.hpi.isg.mdms.domain.constraints.NumberColumnStatistics

// mds.getConstraintCollections.asScala.map(_.getConstraints.asScala)

constraintColl.count
// constraintColl.constraintsIter.foreach { c =}
// constraintColl.allOfType(classOf[NumberColumnStatistics]).size

// constraintColl.groupByType.foreach{ case (key, value) => println(s"$key: ${value.size}") }
// constraintColl.groupByType.foreach { kv => println(s"${kv._1.getSimpleName}: ${kv._2.size}") }

13:35:34.611 [pool-4-thread-2] DEBUG de.hpi.isg.mdms.db.write.BatchWriter - Flushed 4 statements from SQLExecutor [manipulatedTables=[config], accessedTables=[]] in 48 ms 
13:35:34.622 [pool-4-thread-2] WARN  d.h.i.mdms.rdbms.util.LocationCache - The given class hash code 2121176959 is not known for any registered location Type, did you register your location type in class de.hpi.isg.mdms.rdbms.util.LocationCache
13:35:34.634 [pool-4-thread-2] WARN  d.h.i.mdms.rdbms.util.LocationCache - The given class hash code 2121176959 is not known for any registered location Type, did you register your location type in class de.hpi.isg.mdms.rdbms.util.LocationCache
13:35:34.637 [pool-4-thread-2] WARN  d.h.i.mdms.rdbms.util.LocationCache - The given class hash code 2121176959 is not known for any registered location Type, did you register your location type in class de.hpi.isg.mdms.rdbms.util.LocationCache
13:35:34.644 [pool-4-thread-2] WARN  d.h.i.mdms.rdbms.util.LocationCache - The given class ha

import de.hpi.isg.mdms.analytics.ConstraintImplicits._
import de.hpi.isg.mdms.domain.constraints.NumberColumnStatistics
res2_2: Int = 4363

In [36]:
import de.hpi.isg.mdms.analytics.Visualization._
import de.hpi.isg.mdms.analytics.ConstraintImplicits._

// display.html(initBokeh)
val grouped = constraintColl.groupByType
val namedMap = grouped.map { case(k, v) => k.getSimpleName -> v.size.toDouble }
val histo = createHistogram(namedMap)
val line = createLineGraph(namedMap)
display.html(getBokehPlot(histo))
display.html(getBokehPlot(line))

15:51:38.616 [pool-4-thread-126] DEBUG de.hpi.isg.mdms.db.write.BatchWriter - Flushed 4 statements from SQLExecutor [manipulatedTables=[config], accessedTables=[]] in 59 ms 


import de.hpi.isg.mdms.analytics.Visualization._
import de.hpi.isg.mdms.analytics.ConstraintImplicits._
grouped: Map[Class[_] forSome { type _ <: de.hpi.isg.mdms.model.constraints.Constraint }, Iterable[de.hpi.isg.mdms.model.constraints.Constraint]] = Map(
  class de.hpi.isg.mdms.domain.constraints.ColumnStatistics -> List(
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@a570a57,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@6bc02bc5,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@4ddb2dd6,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@33267337,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@76fef6e5,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@49b0a993,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@737cd359,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@109050b7,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@20b7e084,
    de.hpi.isg.mdms.domain.constraints.ColumnStatistics@36bd76f1,
    de.hpi.isg.